In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
        self.conv1_bn = nn.BatchNorm2d(2)
        self.conv2 = nn.Conv2d(2, 4, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(4)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
          
        self.fc = nn.Sequential(
                    nn.Dropout(p=0.1),
                    nn.Linear(in_features=196, out_features=64),
                    nn.BatchNorm1d(64),
                    nn.ReLU(inplace=True),
                    nn.Dropout(p=0.1),
                    nn.Linear(in_features=64, out_features=32),
                    nn.BatchNorm1d(32),
                    nn.ReLU(inplace=True),
                    nn.Linear(in_features=32, out_features=10))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv1_bn(x)
        x = F.relu(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.conv2_bn(x)
        x = F.relu(x)
        x = self.pool2(x)
        
        x = self.avgpool(x)
        x = x.view((x.shape[0],-1))
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 2, 28, 28]              20
       BatchNorm2d-2            [-1, 2, 28, 28]               4
         MaxPool2d-3            [-1, 2, 14, 14]               0
            Conv2d-4            [-1, 4, 14, 14]              76
       BatchNorm2d-5            [-1, 4, 14, 14]               8
         MaxPool2d-6              [-1, 4, 7, 7]               0
 AdaptiveAvgPool2d-7              [-1, 4, 7, 7]               0
           Dropout-8                  [-1, 196]               0
            Linear-9                   [-1, 64]          12,608
      BatchNorm1d-10                   [-1, 64]             128
             ReLU-11                   [-1, 64]               0
          Dropout-12                   [-1, 64]               0
           Linear-13                   [-1, 32]           2,080
      BatchNorm1d-14                   

In [6]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
# for lr in np.linspace(0.0029, 0.003, 5):
#     lr = float(lr)
#     print("== RUNNING LR == ", str(lr))

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0029, momentum=0.9)

for epoch in range(1, 18):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.27429908514022827 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 31.11it/s]



Test set: Average loss: 0.2221, Accuracy: 9499/10000 (94.99%)



loss=0.24828684329986572 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.96it/s]



Test set: Average loss: 0.1185, Accuracy: 9676/10000 (96.76%)



loss=0.07668949663639069 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.17it/s]


...


Test set: Average loss: 0.0435, Accuracy: 9851/10000 (98.51%)



loss=0.1124730184674263 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.24it/s]



Test set: Average loss: 0.0440, Accuracy: 9846/10000 (98.46%)



loss=0.08353384584188461 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.14it/s]



Test set: Average loss: 0.0430, Accuracy: 9861/10000 (98.61%)



Test set: Average loss: 0.0430, Accuracy: 9861/10000 (98.61%)